# Visión por Computador - Práctica 3
## Cámara, geometría epipolar y reconstrucción estérea

### Estimación de la matriz de una cámara a partir del conjunto de puntos en correspondencias.

##### a) Generar la matriz de una cámara finita P a partir de valores aleatorios en [0,1] . Verificar si representa una cámara finita y en ese caso quedársela.
##### b) Suponer un patrón de puntos del mundo 3D compuesto por el conjunto de puntos con coordenadas {(0,x1,x2) y (x2,x1,0), para x1=0.1:0.1:1 y x2=0.1:0.1:1}. Esto supone una rejilla de puntos en dos planos distintos ortogonales.
##### c) Proyectar el conjunto de puntos del mundo con la cámara simulada y obtener las coordenadas píxel de su proyección.
##### d) Implementar el algoritmo DLT para estimar la cámara P a partir de los puntos 3D y sus proyecciones en la imagen.
##### e) Calcular el error de la estimación usando la norma de Frobenius (cuadrática).
##### f) Mostrar en una única imagen los puntos 3D proyectados con la cámara estimada y la cámara simulada.

In [9]:
import cv2
import math
import numpy as np
import functions as fx

def generate_Pcamera():
    # La matriz cámara tiene la siguiente estructura:
    #              P =  K[R | T]
    # donde det(R) != 0, al igual que det(K) != 0, por
    # lo que podemos hacer que P = [KR | KT] = [M | M_4]

    # Generamos una matriz con valores aleatorios en el
    # intervalo [0,1)
    P_cam = np.random.rand(3,4)

    # Comprobamos si det(M) != 0. En caso de que no lo
    # sea, volvemos a generar una nueva matriz cámara.
    while not np.linalg.det(P[0:3,0:3]):
        P_cam = np.random.rand(3,4)        
    
    return P_cam

def generate_points():
    # Generamos los valores de x1 y x2
    x1 = np.arange(start = 0, stop = 1,
                   step = 0.1, dtype=np.float32)
    x2 = np.arange(start = 0, stop = 1,
                   step = 0.1, dtype=np.float32)

    points2D = np.concatenate(np.array(np.meshgrid(x1,x2)).T,np.zeros(x1.shape[0]))

    set1 = np.hstack((np.zeros(points2D.shape[0]).T, conjunto))
    set2 = np.hstack((np.zeros(conjunto, points2D.shape[0]).T))

    return set1, set2